In [4]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import os
import py_entitymatching as em
import pandas_profiling

In [52]:
with open("test.csv") as file:
    test_data = pd.read_csv(file)
with open("ltable.csv") as file:
    ltable = pd.read_csv(file)
with open("rtable.csv") as file:
    rtable = pd.read_csv(file)
with open("train.csv") as file:
    train_data = pd.read_csv(file)
train_data.shape

(7149, 4)

In [6]:
def to_utf8(input_file, output_file=None, write_mode = 'x'):
    if not output_file:
        file, extension = os.path.splitext(input_file)
        output_file = file + '_utf8' + extension
    with open(input_file) as old:
        content= old.read()
    with open(output_file, mode=write_mode,encoding='utf-8') as new:
        new.write(content)
    return output_file

In [7]:
A = em.read_csv_metadata(to_utf8('ltable.csv', write_mode='w'),key='ltable_id')
B = em.read_csv_metadata(to_utf8('rtable.csv', write_mode='w'),key='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [24]:
ob = em.OverlapBlocker()
C = ob.block_tables(A, B, 'title', 'title', 
                    l_output_attrs=[ 'ltable_id', 'title','category', 'brand', 'modelno'],
                    r_output_attrs=[ 'rtable_id', 'title','category', 'brand', 'modelno'],
                    overlap_size=1, show_progress=True)

C:\Users\John\Anaconda3\lib\site-packages\pandas\core\generic.py:4384: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
C:\Users\John\Anaconda3\lib\site-packages\pandas\core\generic.py:4385: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


In [9]:
C.head()

,_id,ltable_ltable_id,rtable_rtable_id,ltable_title,ltable_category,ltable_brand,ltable_modelno,rtable_title,rtable_category,rtable_brand,rtable_modelno
0,0,70,0,panasonic stereo earbud headphones rp-hv21 green,electronics - general,panasonic,rphv21g,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132
1,1,105,0,koss stereophones with flexible headband design,electronics - general,koss,sporta-pro,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132
2,2,293,0,coby deep base stereo headphones cv121,electronics - general,coby,cv121,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132
3,3,490,0,coby pink high-performance isolation stereo earphones cvem79pnk,electronics - general,coby,cvem79pnk,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132
4,4,514,0,ihome portable stereo alarm clock with ipod dock gunmetal,electronics - general,ihome,ih16g,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132


In [27]:
C[['ltable_ltable_id', 'rtable_rtable_id']].isin(train_data[['ltable_id', 'rtable_id']]).any(axis=1).describe()

count     7478010
unique          1
top         False
freq      7478010
dtype: object

In [35]:
D =C[C.ltable_ltable_id == 621]
D[D.rtable_rtable_id == 3167]

,_id,ltable_ltable_id,rtable_rtable_id,ltable_title,ltable_category,ltable_brand,ltable_modelno,rtable_title,rtable_category,rtable_brand,rtable_modelno
1101087,1101087,621,3167,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card,electronics - general,zotac,zt-40604-10l,evga geforce gts450 superclocked 1 gb gddr5 pci-express 2.0 graphics card 01g-p3-1452-tr,graphics cards,evga,01g-p3-1452-tr


In [40]:
C=C.rename(index=str, columns={"ltable_ltable_id": "ltable_id", "rtable_rtable_id": "rtable_id"})

In [41]:
C.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_category,ltable_brand,ltable_modelno,rtable_title,rtable_category,rtable_brand,rtable_modelno
0,0,70,0,panasonic stereo earbud headphones rp-hv21 green,electronics - general,panasonic,rphv21g,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132
1,1,105,0,koss stereophones with flexible headband design,electronics - general,koss,sporta-pro,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132
2,2,293,0,coby deep base stereo headphones cv121,electronics - general,coby,cv121,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132
3,3,490,0,coby pink high-performance isolation stereo earphones cvem79pnk,electronics - general,coby,cvem79pnk,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132
4,4,514,0,ihome portable stereo alarm clock with ipod dock gunmetal,electronics - general,ihome,ih16g,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132


In [42]:
D = C.merge(train_data, on=['ltable_id', 'rtable_id'], how='outer')

In [ ]:
E = D

In [43]:
D.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_category,ltable_brand,ltable_modelno,rtable_title,rtable_category,rtable_brand,rtable_modelno,label,id
0,0,70,0,panasonic stereo earbud headphones rp-hv21 green,electronics - general,panasonic,rphv21g,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,NaN,NaN
1,1,105,0,koss stereophones with flexible headband design,electronics - general,koss,sporta-pro,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,NaN,NaN
2,2,293,0,coby deep base stereo headphones cv121,electronics - general,coby,cv121,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,NaN,NaN
3,3,490,0,coby pink high-performance isolation stereo earphones cvem79pnk,electronics - general,coby,cvem79pnk,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,NaN,NaN
4,4,514,0,ihome portable stereo alarm clock with ipod dock gunmetal,electronics - general,ihome,ih16g,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,NaN,NaN


In [68]:
D = D.dropna(subset=['label'])
D = D.astype({"label": int, "id": int})
D

,_id,ltable_id,rtable_id,ltable_title,ltable_category,ltable_brand,ltable_modelno,rtable_title,rtable_category,rtable_brand,rtable_modelno,label,id
339,339,2069,1,kodak 10c color ink cartridge,printers,kodak,10c,kodak black ink cartridge 10b 1163641,inkjet printer ink,kodak,1163641,0,9766
1501,1501,1197,4,kensington k72335us ci10 fit wireless notebook laser mouse,mice,kensington,k72335us,kensington k72349us wireless mouse for netbooks,mice,kensington,k72349us,0,7579
2687,2687,1036,8,maxell 635128 8x dvd-r media,mp3 accessories,maxell,NaN,16x dvd-r media,dvd-r discs,imation,17340,0,3952
2697,2697,1909,8,verbatim 95137 16x dvd-r media,mp3 accessories,verbatim,95137,16x dvd-r media,dvd-r discs,imation,17340,0,6082
3536,3536,2405,12,hp laserjet 53x black print cartridge,printers,hp,q7553x,hp 85a laserjet black toner print cartridge - retail packaging ce285a,laser printer toner,hp,ce285a,0,2278
5711,5711,391,20,energizer max aaa 6pk,electronics - general,energizer,e92bp-6r,energizer max alkaline batteries,d,energizer,e95fp-8,0,8194
9293,9293,419,30,western digital 1tb my passport essential se usb 3.0 portable external hard drive,hard drives,western digital,wdbacx0010bbksn,western digital my book essential 1 tb usb 3.0 2.0 desktop external hard drive,external hard drives,western digital,wdbacw0010hbk-nesn,0,3769
9297,9297,429,30,western digital 2tb my book studio edition ii external hard drive,computers,western digital,wdh2q20000n,western digital my book essential 1 tb usb 3.0 2.0 desktop external hard drive,external hard drives,western digital,wdbacw0010hbk-nesn,0,1075
9415,9415,998,30,western digital av-gp 320gb sata desktop internal hard drive,hard drives,western digital,wd3200avvs,western digital my book essential 1 tb usb 3.0 2.0 desktop external hard drive,external hard drives,western digital,wdbacw0010hbk-nesn,0,5564
9671,9671,2394,30,western digital 750gb my passport essential se usb 3.0 silver portable hard drive,hard drives,western digital,wdbacx7500asl-nesn,western digital my book essential 1 tb usb 3.0 2.0 desktop external hard drive,external hard drives,western digital,wdbacw0010hbk-nesn,0,6305


In [65]:
path_G = 'df_d.csv'
D.to_csv(path_G)
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [69]:
G

,Unnamed: 0,_id,ltable_id,rtable_id,ltable_title,ltable_category,ltable_brand,ltable_modelno,rtable_title,rtable_category,rtable_brand,rtable_modelno,label,id
0,339,339,2069,1,kodak 10c color ink cartridge,printers,kodak,10c,kodak black ink cartridge 10b 1163641,inkjet printer ink,kodak,1163641,0,9766
1,1501,1501,1197,4,kensington k72335us ci10 fit wireless notebook laser mouse,mice,kensington,k72335us,kensington k72349us wireless mouse for netbooks,mice,kensington,k72349us,0,7579
2,2687,2687,1036,8,maxell 635128 8x dvd-r media,mp3 accessories,maxell,NaN,16x dvd-r media,dvd-r discs,imation,17340,0,3952
3,2697,2697,1909,8,verbatim 95137 16x dvd-r media,mp3 accessories,verbatim,95137,16x dvd-r media,dvd-r discs,imation,17340,0,6082
4,3536,3536,2405,12,hp laserjet 53x black print cartridge,printers,hp,q7553x,hp 85a laserjet black toner print cartridge - retail packaging ce285a,laser printer toner,hp,ce285a,0,2278
5,5711,5711,391,20,energizer max aaa 6pk,electronics - general,energizer,e92bp-6r,energizer max alkaline batteries,d,energizer,e95fp-8,0,8194
6,9293,9293,419,30,western digital 1tb my passport essential se usb 3.0 portable external hard drive,hard drives,western digital,wdbacx0010bbksn,western digital my book essential 1 tb usb 3.0 2.0 desktop external hard drive,external hard drives,western digital,wdbacw0010hbk-nesn,0,3769
7,9297,9297,429,30,western digital 2tb my book studio edition ii external hard drive,computers,western digital,wdh2q20000n,western digital my book essential 1 tb usb 3.0 2.0 desktop external hard drive,external hard drives,western digital,wdbacw0010hbk-nesn,0,1075
8,9415,9415,998,30,western digital av-gp 320gb sata desktop internal hard drive,hard drives,western digital,wd3200avvs,western digital my book essential 1 tb usb 3.0 2.0 desktop external hard drive,external hard drives,western digital,wdbacw0010hbk-nesn,0,5564
9,9671,9671,2394,30,western digital 750gb my passport essential se usb 3.0 silver portable hard drive,hard drives,western digital,wdbacx7500asl-nesn,western digital my book essential 1 tb usb 3.0 2.0 desktop external hard drive,external hard drives,western digital,wdbacw0010hbk-nesn,0,6305


In [70]:
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [71]:
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,category_category_jac_qgm_3_qgm_3,category,category,qgm_3,qgm_3,jaccard,<function category_category_jac_qgm_3_qgm_3 at 0x000001884B934378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,category_category_cos_dlm_dc0_dlm_dc0,category,category,dlm_dc0,dlm_dc0,cosine,<function category_category_cos_dlm_dc0_dlm_dc0 at 0x000001884B9342F0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,category_category_jac_dlm_dc0_dlm_dc0,category,category,dlm_dc0,dlm_dc0,jaccard,<function category_category_jac_dlm_dc0_dlm_dc0 at 0x000001884BFD9598>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,category_category_mel,category,category,None,None,monge_elkan,<function category_category_mel at 0x000001884BFD9A60>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,category_category_lev_dist,category,category,None,None,lev_dist,<function category_category_lev_dist at 0x000001884BFD9AE8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,category_category_lev_sim,category,category,None,None,lev_sim,<function category_category_lev_sim at 0x000001884BFD9B70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,category_category_nmw,category,category,None,None,needleman_wunsch,<function category_category_nmw at 0x000001884BFD9D08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,category_category_sw,category,category,None,None,smith_waterman,<function category_category_sw at 0x000001884BFD9C80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,brand_brand_jac_qgm_3_qgm_3,brand,brand,qgm_3,qgm_3,jaccard,<function brand_brand_jac_qgm_3_qgm_3 at 0x000001884BFD9D90>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,brand_brand_cos_dlm_dc0_dlm_dc0,brand,brand,dlm_dc0,dlm_dc0,cosine,<function brand_brand_cos_dlm_dc0_dlm_dc0 at 0x000001884BFD9EA0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [72]:
D.columns

Index(['_id', 'ltable_id', 'rtable_id', 'ltable_title', 'ltable_category',
       'ltable_brand', 'ltable_modelno', 'rtable_title', 'rtable_category',
       'rtable_brand', 'rtable_modelno', 'label', 'id'],
      dtype='object')

In [91]:
# Select the attrs. to be included in the feature vector table
# attrs_from_table = ['ltable_title', 'ltable_brand', 'ltable_modelno',
#                    'rtable_title', 'rtable_brand', 'rtable_modelno']
attrs_from_table = ['ltable_title', 'rtable_title']
# Convert the labeled data to feature vectors using the feature table
H = em.extract_feature_vecs(G, 
                            feature_table=feature_table, 
                            attrs_before = attrs_from_table,
                            attrs_after='label',
                            show_progress=False)

In [92]:
# Instantiate the RF Matcher
rf = em.RFMatcher()

In [93]:
# Get the attributes to be projected while training
attrs_to_be_excluded = []
attrs_to_be_excluded.extend(['_id', 'ltable_id', 'rtable_id', 'label'])
attrs_to_be_excluded.extend(attrs_from_table)

In [94]:
H.columns

Index(['_id', 'ltable_id', 'rtable_id', 'ltable_title', 'rtable_title',
       'category_category_jac_qgm_3_qgm_3',
       'category_category_cos_dlm_dc0_dlm_dc0',
       'category_category_jac_dlm_dc0_dlm_dc0', 'category_category_mel',
       'category_category_lev_dist', 'category_category_lev_sim',
       'category_category_nmw', 'category_category_sw',
       'brand_brand_jac_qgm_3_qgm_3', 'brand_brand_cos_dlm_dc0_dlm_dc0',
       'brand_brand_jac_dlm_dc0_dlm_dc0', 'brand_brand_mel',
       'brand_brand_lev_dist', 'brand_brand_lev_sim', 'brand_brand_nmw',
       'brand_brand_sw', 'modelno_modelno_jac_qgm_3_qgm_3',
       'modelno_modelno_cos_dlm_dc0_dlm_dc0',
       'modelno_modelno_jac_dlm_dc0_dlm_dc0', 'modelno_modelno_mel',
       'modelno_modelno_lev_dist', 'modelno_modelno_lev_sim',
       'modelno_modelno_nmw', 'modelno_modelno_sw', 'price_price_exm',
       'price_price_anm', 'price_price_lev_dist', 'price_price_lev_sim',
       'label'],
      dtype='object')

In [95]:
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], strategy='mean')

ValueError: could not convert string to float: 'kodak 10c color ink cartridge'

In [96]:
# Train using feature vectors from the labeled data.
rf.fit(table=H, exclude_attrs=attrs_to_be_excluded, target_attr='label')

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').